# Module 4.4: SVM - 支持向量機

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解 SVM 的幾何直覺（最大間隔分類器）
2. 理解 Hinge Loss 的定義與性質
3. 使用 Subgradient Descent 優化 SVM
4. 視覺化支持向量與決策邊界

## 背景知識

SVM (Support Vector Machine) 是一種強大的分類器，核心想法是：

> 找一個超平面，使得兩類資料之間的「間隔」最大化

---

## 參考資源

- Bishop PRML Ch. 7: Sparse Kernel Machines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("環境設定完成！")

---

## Part 1: SVM 的幾何直覺

### 線性分類器回顧

決策函數：$f(x) = w^T x + b$

- $f(x) > 0$：預測類別 +1
- $f(x) < 0$：預測類別 -1

### 間隔（Margin）

點 $x$ 到超平面的距離：

$$\text{distance} = \frac{|w^T x + b|}{\|w\|}$$

**間隔** = 最近點到超平面的距離

### SVM 目標

最大化間隔，同時正確分類所有點：

$$\max_{w, b} \frac{2}{\|w\|} \quad \text{s.t.} \quad y_i (w^T x_i + b) \geq 1$$

In [ ]:
# 生成線性可分資料
def generate_linearly_separable_data(n_samples=100, margin=1.0, seed=42):
    """
    生成線性可分的二分類資料
    
    Parameters
    ----------
    margin : float
        兩類之間的最小距離
    """
    np.random.seed(seed)
    
    n_per_class = n_samples // 2
    
    # 類別 -1
    X_neg = np.random.randn(n_per_class, 2) * 0.8
    X_neg[:, 0] -= margin
    
    # 類別 +1
    X_pos = np.random.randn(n_per_class, 2) * 0.8
    X_pos[:, 0] += margin
    
    X = np.vstack([X_neg, X_pos])
    y = np.array([-1] * n_per_class + [1] * n_per_class)
    
    idx = np.random.permutation(n_samples)
    return X[idx], y[idx]


# 視覺化
X, y = generate_linearly_separable_data(n_samples=100, margin=1.5)

plt.figure(figsize=(10, 8))
plt.scatter(X[y==-1, 0], X[y==-1, 1], c='blue', marker='o', s=50, label='Class -1')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='s', s=50, label='Class +1')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('線性可分資料')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

---

## Part 2: Hinge Loss

### 定義

對於標籤 $y \in \{-1, +1\}$：

$$L_{\text{hinge}}(y, f(x)) = \max(0, 1 - y \cdot f(x))$$

其中 $f(x) = w^T x + b$

### 直覺

- 如果 $y \cdot f(x) \geq 1$（正確分類且距離超平面夠遠），loss = 0
- 如果 $y \cdot f(x) < 1$，loss = $1 - y \cdot f(x)$

### SVM 優化問題

$$\min_{w, b} \frac{1}{N} \sum_{i=1}^N \max(0, 1 - y_i (w^T x_i + b)) + \frac{\lambda}{2} \|w\|^2$$

這是 Hinge Loss + L2 正則化。

In [ ]:
def hinge_loss(y_true, scores):
    """
    計算 Hinge Loss
    
    Parameters
    ----------
    y_true : np.ndarray, shape (N,)
        真實標籤，值為 -1 或 +1
    scores : np.ndarray, shape (N,)
        決策函數的輸出 f(x) = w^T x + b
    
    Returns
    -------
    float
        平均 Hinge Loss
    """
    margins = y_true * scores  # y * f(x)
    losses = np.maximum(0, 1 - margins)  # max(0, 1 - y*f(x))
    return np.mean(losses)


# 視覺化 Hinge Loss vs 其他 Loss
scores = np.linspace(-3, 3, 100)
y = 1  # 假設真實標籤是 +1

# 不同的 loss 函數
hinge = np.maximum(0, 1 - y * scores)
logistic = np.log(1 + np.exp(-y * scores))  # Logistic loss
zero_one = (y * scores < 0).astype(float)  # 0-1 loss

plt.figure(figsize=(10, 6))
plt.plot(scores, hinge, 'b-', linewidth=2, label='Hinge Loss')
plt.plot(scores, logistic, 'g-', linewidth=2, label='Logistic Loss')
plt.plot(scores, zero_one, 'r--', linewidth=2, label='0-1 Loss')
plt.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
plt.axvline(x=1, color='gray', linestyle=':', alpha=0.5, label='Margin boundary')
plt.xlabel('$f(x) = w^T x + b$ (for y=+1)')
plt.ylabel('Loss')
plt.title('不同 Loss 函數的比較')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(-3, 3)
plt.ylim(-0.5, 4)
plt.show()

print("觀察：")
print("- Hinge Loss 在 f(x) >= 1 時為 0（滿足 margin）")
print("- Hinge Loss 是 0-1 Loss 的凸上界")
print("- Logistic Loss 永遠不為 0，但比較平滑")

---

## Part 3: Subgradient Descent

### 問題

Hinge Loss 在 $1 - y \cdot f(x) = 0$ 處不可微分（有折點）。

### Subgradient

對於 $\max(0, z)$：

$$\frac{\partial \max(0, z)}{\partial z} = \begin{cases} 0 & \text{if } z < 0 \\ 1 & \text{if } z > 0 \\ [0, 1] & \text{if } z = 0 \end{cases}$$

實作中，$z = 0$ 時任選 0 或 1。

### SVM 梯度

令 $L_i = \max(0, 1 - y_i (w^T x_i + b))$

$$\frac{\partial L_i}{\partial w} = \begin{cases} 0 & \text{if } y_i (w^T x_i + b) \geq 1 \\ -y_i x_i & \text{otherwise} \end{cases}$$

$$\frac{\partial L_i}{\partial b} = \begin{cases} 0 & \text{if } y_i (w^T x_i + b) \geq 1 \\ -y_i & \text{otherwise} \end{cases}$$

加上正則化項：

$$\frac{\partial L}{\partial w} = \frac{1}{N} \sum_i \frac{\partial L_i}{\partial w} + \lambda w$$

In [ ]:
def compute_svm_gradients(X, y, w, b, reg=0.01):
    """
    計算 SVM 的 subgradients
    
    Parameters
    ----------
    X : np.ndarray, shape (N, D)
    y : np.ndarray, shape (N,)
        標籤為 -1 或 +1
    w : np.ndarray, shape (D,)
    b : float
    reg : float
        正則化參數 λ
    
    Returns
    -------
    dw : np.ndarray, shape (D,)
    db : float
    """
    N, D = X.shape
    
    # 計算 margin
    scores = X @ w + b  # shape (N,)
    margins = y * scores  # y * f(x)
    
    # 找出違反 margin 的樣本
    # margin < 1 表示有 loss
    violating = margins < 1  # shape (N,) boolean
    
    # 計算梯度
    # 對於違反 margin 的樣本：∂L/∂w = -y*x, ∂L/∂b = -y
    # 對於滿足 margin 的樣本：梯度為 0
    
    dw = np.zeros(D)
    db = 0.0
    
    for i in range(N):
        if violating[i]:
            dw += -y[i] * X[i]
            db += -y[i]
    
    dw = dw / N + reg * w  # 平均 + 正則化
    db = db / N  # bias 不正則化
    
    return dw, db


# 向量化版本（更快）
def compute_svm_gradients_vectorized(X, y, w, b, reg=0.01):
    """
    向量化的 SVM subgradient 計算
    """
    N = X.shape[0]
    
    scores = X @ w + b
    margins = y * scores
    
    # 違反 margin 的樣本
    violating = (margins < 1).astype(float)  # shape (N,)
    
    # 梯度（向量化）
    # dw = -1/N * sum_{violating} y_i * x_i + λw
    dw = -(1/N) * (X.T @ (violating * y)) + reg * w
    db = -(1/N) * np.sum(violating * y)
    
    return dw, db


# 驗證兩個版本一致
np.random.seed(42)
X_test = np.random.randn(50, 3)
y_test = np.sign(np.random.randn(50))
w_test = np.random.randn(3)
b_test = np.random.randn()

dw1, db1 = compute_svm_gradients(X_test, y_test, w_test, b_test)
dw2, db2 = compute_svm_gradients_vectorized(X_test, y_test, w_test, b_test)

print("=== 驗證梯度實作 ===")
print(f"dw 差異: {np.linalg.norm(dw1 - dw2):.10f}")
print(f"db 差異: {abs(db1 - db2):.10f}")

---

## Part 4: 完整 SVM 實作

In [ ]:
class LinearSVM:
    """
    Linear SVM 使用 Hinge Loss + L2 正則化
    
    優化問題：
    min (1/N) Σ max(0, 1 - y_i(w·x_i + b)) + (λ/2)||w||²
    
    Parameters
    ----------
    C : float
        正則化參數。C = 1/λ，較大的 C 表示較少正則化
    """
    
    def __init__(self, C=1.0):
        self.C = C
        self.reg = 1.0 / C  # λ = 1/C
        self.w = None
        self.b = None
        self.history = None
    
    def _compute_loss(self, X, y):
        """計算總 loss（hinge + regularization）"""
        scores = X @ self.w + self.b
        margins = y * scores
        hinge = np.mean(np.maximum(0, 1 - margins))
        reg_loss = 0.5 * self.reg * np.sum(self.w ** 2)
        return hinge + reg_loss
    
    def fit(self, X, y, lr=0.01, n_iter=1000, verbose=True):
        """
        使用 Subgradient Descent 訓練 SVM
        
        Parameters
        ----------
        X : np.ndarray, shape (N, D)
        y : np.ndarray, shape (N,)
            標籤為 -1 或 +1
        lr : float
            學習率
        n_iter : int
            迭代次數
        """
        N, D = X.shape
        
        # 確保標籤是 -1 或 +1
        y = np.where(y == 0, -1, y)
        
        # 初始化
        self.w = np.zeros(D)
        self.b = 0.0
        self.history = {'loss': [], 'accuracy': []}
        
        for i in range(n_iter):
            # 計算梯度
            scores = X @ self.w + self.b
            margins = y * scores
            violating = (margins < 1).astype(float)
            
            dw = -(1/N) * (X.T @ (violating * y)) + self.reg * self.w
            db = -(1/N) * np.sum(violating * y)
            
            # 更新
            self.w = self.w - lr * dw
            self.b = self.b - lr * db
            
            # 記錄
            loss = self._compute_loss(X, y)
            predictions = np.sign(X @ self.w + self.b)
            accuracy = np.mean(predictions == y)
            
            self.history['loss'].append(loss)
            self.history['accuracy'].append(accuracy)
            
            if verbose and (i + 1) % (n_iter // 10) == 0:
                print(f"Iter {i+1:4d}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
        
        return self
    
    def predict(self, X):
        """預測類別（-1 或 +1）"""
        scores = X @ self.w + self.b
        return np.sign(scores)
    
    def decision_function(self, X):
        """返回決策分數 f(x) = w·x + b"""
        return X @ self.w + self.b
    
    def score(self, X, y):
        """計算準確率"""
        y = np.where(y == 0, -1, y)
        predictions = self.predict(X)
        return np.mean(predictions == y)
    
    def get_support_vectors(self, X, y, tol=1e-3):
        """
        找出支持向量
        
        支持向量是滿足 |y * f(x) - 1| < tol 的點
        """
        y = np.where(y == 0, -1, y)
        margins = y * self.decision_function(X)
        
        # 支持向量：margin 接近 1 或 margin < 1
        sv_mask = margins <= 1 + tol
        return X[sv_mask], y[sv_mask], np.where(sv_mask)[0]


# 測試
print("=== 測試 Linear SVM ===")

X, y = generate_linearly_separable_data(n_samples=100, margin=1.5)

print(f"資料大小: {X.shape}")
print(f"標籤分布: {np.bincount(((y + 1) / 2).astype(int))}")

# 訓練
svm = LinearSVM(C=1.0)
svm.fit(X, y, lr=0.01, n_iter=1000, verbose=True)

print(f"\n最終準確率: {svm.score(X, y):.4f}")
print(f"權重: w = {svm.w}")
print(f"偏置: b = {svm.b:.4f}")

In [ ]:
# 視覺化 SVM 決策邊界與支持向量

def plot_svm_decision_boundary(svm, X, y, title="SVM Decision Boundary"):
    """
    繪製 SVM 決策邊界、間隔和支持向量
    """
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # 決策分數
    Z = svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 8))
    
    # 決策函數等高線
    plt.contourf(xx, yy, Z, levels=np.linspace(-3, 3, 13), cmap='RdYlBu', alpha=0.4)
    plt.colorbar(label='$f(x) = w^T x + b$')
    
    # 決策邊界 (f=0) 和 margin 邊界 (f=±1)
    plt.contour(xx, yy, Z, levels=[-1, 0, 1], colors=['blue', 'black', 'red'],
                linestyles=['--', '-', '--'], linewidths=[1.5, 2, 1.5])
    
    # 資料點
    y_plot = np.where(y == 0, -1, y)
    plt.scatter(X[y_plot==-1, 0], X[y_plot==-1, 1], c='blue', marker='o', 
                s=50, label='Class -1', edgecolors='k')
    plt.scatter(X[y_plot==1, 0], X[y_plot==1, 1], c='red', marker='s', 
                s=50, label='Class +1', edgecolors='k')
    
    # 支持向量
    sv_X, sv_y, sv_idx = svm.get_support_vectors(X, y)
    plt.scatter(sv_X[:, 0], sv_X[:, 1], s=200, facecolors='none', 
                edgecolors='green', linewidths=2, label=f'Support Vectors ({len(sv_idx)})')
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)


plot_svm_decision_boundary(svm, X, y, title='Linear SVM')
plt.show()

# 訓練曲線
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(svm.history['loss'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss (Hinge + L2)')
axes[0].set_title('訓練 Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(svm.history['accuracy'])
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('訓練準確率')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 5: C 參數的影響

In [ ]:
# C 參數的影響
print("=== C 參數的影響 ===")

# 生成有一些噪音的資料
X_noisy, y_noisy = generate_linearly_separable_data(n_samples=100, margin=0.5, seed=42)

# 加入一些錯分點
np.random.seed(123)
flip_idx = np.random.choice(100, 5, replace=False)
y_noisy[flip_idx] *= -1

# 不同的 C 值
C_values = [0.1, 1.0, 10.0, 100.0]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for ax, C in zip(axes, C_values):
    svm = LinearSVM(C=C)
    svm.fit(X_noisy, y_noisy, lr=0.01, n_iter=1000, verbose=False)
    
    # 繪製
    x_min, x_max = X_noisy[:, 0].min() - 1, X_noisy[:, 0].max() + 1
    y_min, y_max = X_noisy[:, 1].min() - 1, X_noisy[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    Z = svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, levels=np.linspace(-3, 3, 13), cmap='RdYlBu', alpha=0.4)
    ax.contour(xx, yy, Z, levels=[-1, 0, 1], colors=['blue', 'black', 'red'],
               linestyles=['--', '-', '--'], linewidths=[1.5, 2, 1.5])
    
    y_plot = np.where(y_noisy == 0, -1, y_noisy)
    ax.scatter(X_noisy[y_plot==-1, 0], X_noisy[y_plot==-1, 1], c='blue', marker='o', s=50)
    ax.scatter(X_noisy[y_plot==1, 0], X_noisy[y_plot==1, 1], c='red', marker='s', s=50)
    
    sv_X, _, sv_idx = svm.get_support_vectors(X_noisy, y_noisy)
    ax.scatter(sv_X[:, 0], sv_X[:, 1], s=200, facecolors='none', 
               edgecolors='green', linewidths=2)
    
    accuracy = svm.score(X_noisy, y_noisy)
    ax.set_title(f'C = {C}\nAcc: {accuracy:.2f}, SVs: {len(sv_idx)}')
    ax.grid(True, alpha=0.3)

plt.suptitle('不同 C 值的 SVM', fontsize=14)
plt.tight_layout()
plt.show()

print("觀察：")
print("- 較小的 C（強正則化）：間隔較寬，可能欠擬合")
print("- 較大的 C（弱正則化）：間隔較窄，更接近 hard-margin SVM")

---

## 練習題

### 練習 1：Multi-class SVM (One-vs-All)

In [ ]:
# 練習 1 解答

class MultiClassSVM:
    """
    使用 One-vs-All 策略的多分類 SVM
    """
    
    def __init__(self, n_classes, C=1.0):
        self.n_classes = n_classes
        self.C = C
        self.classifiers = []
    
    def fit(self, X, y, lr=0.01, n_iter=1000, verbose=False):
        """
        訓練 K 個二分類 SVM
        """
        self.classifiers = []
        
        for k in range(self.n_classes):
            if verbose:
                print(f"訓練分類器 {k}...")
            
            # 建立二分類標籤：類別 k 為 +1，其他為 -1
            y_binary = np.where(y == k, 1, -1)
            
            # 訓練 SVM
            svm = LinearSVM(C=self.C)
            svm.fit(X, y_binary, lr=lr, n_iter=n_iter, verbose=False)
            
            self.classifiers.append(svm)
        
        return self
    
    def predict(self, X):
        """
        預測：選擇分數最高的類別
        """
        scores = np.zeros((X.shape[0], self.n_classes))
        
        for k, svm in enumerate(self.classifiers):
            scores[:, k] = svm.decision_function(X)
        
        return np.argmax(scores, axis=1)
    
    def score(self, X, y):
        """計算準確率"""
        predictions = self.predict(X)
        return np.mean(predictions == y)


# 測試多分類 SVM
print("=== 測試 Multi-class SVM ===")

# 生成 3 類資料
def generate_3class_data(n_samples=150, seed=42):
    np.random.seed(seed)
    n_per_class = n_samples // 3
    
    X0 = np.random.randn(n_per_class, 2) * 0.6 + np.array([0, 2])
    X1 = np.random.randn(n_per_class, 2) * 0.6 + np.array([-1.5, -1])
    X2 = np.random.randn(n_per_class, 2) * 0.6 + np.array([1.5, -1])
    
    X = np.vstack([X0, X1, X2])
    y = np.array([0] * n_per_class + [1] * n_per_class + [2] * n_per_class)
    
    idx = np.random.permutation(len(y))
    return X[idx], y[idx]


X3, y3 = generate_3class_data(n_samples=150)

mc_svm = MultiClassSVM(n_classes=3, C=1.0)
mc_svm.fit(X3, y3, lr=0.01, n_iter=1000, verbose=True)

print(f"\n準確率: {mc_svm.score(X3, y3):.4f}")

# 視覺化
x_min, x_max = X3[:, 0].min() - 1, X3[:, 0].max() + 1
y_min, y_max = X3[:, 1].min() - 1, X3[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

Z = mc_svm.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
plt.contour(xx, yy, Z, colors='black', linewidths=0.5)

colors = ['blue', 'red', 'green']
for k in range(3):
    mask = y3 == k
    plt.scatter(X3[mask, 0], X3[mask, 1], c=colors[k], s=50, 
               label=f'Class {k}', edgecolors='k')

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Multi-class SVM (One-vs-All)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## 總結

### 本 Notebook 涵蓋的內容

1. **SVM 幾何直覺**：
   - 最大化間隔
   - 支持向量

2. **Hinge Loss**：
   - $L = \max(0, 1 - y \cdot f(x))$
   - 分段線性，是 0-1 loss 的凸上界

3. **Subgradient Descent**：
   - 處理不可微分的 loss
   - 梯度選擇

4. **C 參數**：
   - C = 1/λ
   - 控制正則化強度

### 關鍵要點

1. SVM 的優點：最大化間隔有理論保證
2. Hinge Loss 的優點：sparse solutions（很多點的梯度為 0）
3. 支持向量：決定決策邊界的關鍵點

### 下一步

在下一個 notebook 中，我們將學習 **K-Means 聚類**。